Perfect. Here's a full **implementation plan** to build an AI trading system that mirrors how professional traders operate — including **statistical validation and confidence estimation** of its performance.

---

## 🧠 Goal

Build a **self-aware AI trading system** that can:

1. Specialize and trade only in predictable environments.
2. Adapt across market regimes.
3. Monitor its own edge and deactivate when performance decays.
4. Backtest and validate statistically that it adds value beyond random chance.

---

## 🔧 Implementation Blueprint

### 📦 1. **Data Preparation**

#### A. Split Market into Rolling Windows

* Train: `T_train` (e.g., 2 months)
* Test: `T_test` (next month)
* Repeat walkforward stock-by-stock

#### B. Feature Engineering (per stock)

* Technical indicators (volatility, momentum)
* Chaos indicators (entropy, Hurst, recurrence)
* Regime labels (from market clustering or HMM)
* Sentiment (optional)

---

### 🤖 2. **Agent Training per Episode**

Train **PPO** and optionally **A2C** using:

* Our `RecurrentPPO + TransformerPolicy`
* `SequenceAwareNormAbsMoveEnv`
* Early stopping if reward plateaus
* Use `Monitor` wrapper to track episode stats

---

### 🏷️ 3. **Label Advantage for Predictability**

Evaluate each stock-month:

* Compute agent reward vs. random agent:

  $$
  \text{Advantage} = \mathbb{E}[R_{\text{agent}} - R_{\text{random}}]
  $$
* Run **t-test** and **Mann-Whitney U test**:

  * `p_value < 0.05` ⇒ statistically significant edge
  * Log these in the metadata

---

### ⚠️ 4. **Only Keep Statistically Significant Runs**

* Filter episodes where agent significantly beats random.
* Label those stock-months as **"predictable."**
* Store mean/variance/confidence intervals for advantage.

---

### 📈 5. **Meta-Model: Predictable Environment Classifier**

* Train a contrastive classifier:

  * Input: Meta-features (residuals, entropy, etc.)
  * Target: A > B if `Advantage_A > Advantage_B`
* Use:

  * Logistic Regression or LightGBM
  * CV-AUC as performance metric
  * Optionally include `Agent Agreement Score` (PPO vs A2C)

---

### 🔁 6. **Walk-Forward Pipeline**

For each window:

1. **Train agents** and evaluate advantage
2. **Update predictability model**
3. **Predict where to deploy agent next window**
4. **Deploy agent only in predicted-advantageous environments**

---

### 📊 7. **Statistical Confidence System**

Every time a trade decision is made, store:

* Current episode meta-features
* Agent's predicted advantage
* Historical advantage CI from similar episodes
* Disagreement between PPO vs A2C

This enables confidence scores like:

```python
if p_value < 0.01 and agent_agreement > 0.8:
    trust_score = "HIGH"
elif p_value < 0.05:
    trust_score = "MODERATE"
else:
    trust_score = "LOW"
```

---

### 🧾 8. **Evaluation Metrics per Walk**

For each test window:

* Sharpe ratio vs. random
* Hit rate (profitable trades / total)
* Advantage significance count
* Meta-model AUC / accuracy
* Breakdown by regime

We store these in a dataframe for long-term system health monitoring.

---

### ✅ Optional Diagnostics

* **Visualize attention weights** of Transformer to explain decisions.
* **Track per-agent equity curve** and max drawdown.
* Highlight statistically unreliable runs (p > 0.05) in red.

---

## 📁 Output Artifacts

| File                       | Description                                  |
| -------------------------- | -------------------------------------------- |
| `advantage_labels.pkl`     | Agent vs Random reward stats per episode     |
| `meta_features.csv`        | Per stock-month feature set                  |
| `predictability_model.pkl` | Trained contrastive classifier               |
| `walkforward_report.md`    | Summary of all periods with confidence stats |
| `trust_scores.csv`         | Deployment period trust flags                |

---

## ✅ Final Benefits

This system behaves like a **professional trader**:

* Only trades when it knows it can win
* Tracks when it’s losing edge
* Has statistical proof to back up decisions
* Can say “I don’t know” and step out
* Improves itself over time

---

Would you like this pipeline turned into executable Python code? I can build it modular, resumable, and log everything per window/stock.


In [87]:
import jupyter

In [88]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.utils.system import boot
from src.defaults import TOP2_STOCK_BY_SECTOR, RANDOM_SEEDS
from src.data.feature_pipeline import load_base_dataframe

DEVICE = boot()
OHLCV_DF = load_base_dataframe()

In [144]:
from hurst import compute_Hc
from pyts.image import RecurrencePlot

# PROJECT SETUP ==================================
TICKERS = OHLCV_DF['symbol'].unique()#TOP2_STOCK_BY_SECTOR #["TSLA"]
CHAOS_THRESHOLD = 0.45
WINDOW_SIZE = 120
ENTROPY_BINS=10
STEP_SIZE = 20
MAX_LEN = 64
LOOKBACK= 40
ROLLING = 5
SEEDS = RANDOM_SEEDS[:5]
BASIC_FEATURES=["close"	, #Core price for reward and trend awareness
"volume",#	Volume for activity level
"candle_body",#	Price strength (close-open)
"upper_shadow",#	Wick size = volatility / exhaustion
"lower_shadow",#	Same as above
"order_flow",#	Flow = pressure indicator (buy/sell imbalance)
"price_change",#	Price momentum short term
"volatility",#	Recent price dispersion
"momentum",#	Rolling price trend
"vix_norm",#	Implied market risk normalized
"market_return_1d"#	Market regime alignment
]

# That’s 11 columns, enough to:
# * See price movement
# * Detect regime shifts
# * Respond to risk

# Second Round:
# * overnight_price_change → if overnight gaps matter to your strategy
# * trade_count_change → intraday activity shifts
# * sp500_norm → macro regime normalization




In [269]:
# Step 1 - walkforward
import pandas as pd
import ace_tools_open as tools
from typing import List, Tuple
from dateutil.relativedelta import relativedelta
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import RobustScaler



def generate_walkforward_windows(
    df: pd.DataFrame,
    symbol_col: str = "symbol",
    date_col: str = "date",
    start_date: str = "2023-01-01",
    end_date: str = "2025-05-01",
    train_months: int = 2,
    test_months: int = 1,
    min_days_per_window: int = 20,
    max_feature_rolling_window:int=5,
    lookback_buffer_days: int = 30

) -> List[dict]:
    """
    Splits a dataframe into walkforward windows (train and test) per symbol.

    Parameters:
        df: Full OHLCV dataframe with at least [date_col, symbol_col]
        symbol_col: Name of the symbol column
        date_col: Name of the date column
        start_date: Start date of walkforward (string or datetime)
        end_date: End date of walkforward (string or datetime)
        train_months: Number of months in training window
        test_months: Number of months in test window
        min_days_per_window: Minimum number of rows to consider a window valid

    Returns:
        List of window dicts with keys:
            - symbol
            - train_start
            - train_end
            - test_start
            - test_end
            - train_df
            - test_df
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    windows = []
    symbols = df[symbol_col].unique()

    for symbol in symbols:
        symbol_df = df[df[symbol_col] == symbol].sort_values(date_col).reset_index()
        current_start = start_date

        while True:
            # Train window with padded lookback
            episode_start = current_start
            buffer_size = lookback_buffer_days + max_feature_rolling_window
            episode_idx = symbol_df[symbol_df[date_col] >= current_start].index[0]
            #print(current_start,episode_idx)
            train_start_idx = episode_idx - buffer_size

            if train_start_idx < 0:
                current_start = current_start + relativedelta(months=1)  # skip this episode
                continue

            train_start = symbol_df.iloc[train_start_idx][date_col]
            train_end = episode_start + relativedelta(months=train_months)
            
            
            test_start_idx = symbol_df[symbol_df[date_col] >= train_end].index[0] - buffer_size
            
            if test_start_idx < 0:
                break

            test_start = symbol_df.iloc[test_start_idx][date_col]
            test_end = train_end + relativedelta(months=test_months)
            
            if test_end > end_date:
                break

            train_df = symbol_df[(symbol_df[date_col] >= train_start) & (symbol_df[date_col] < train_end)]
            test_df = symbol_df[(symbol_df[date_col] >= test_start) & (symbol_df[date_col] < test_end)]
            #print(len(train_df),len(test_df))
            #print(len(test_df))
            if len(train_df) >= min_days_per_window and len(test_df) >= min_days_per_window:
                windows.append({
                    "symbol": symbol,
                    "train_start": train_start,
                    "train_end": train_end,
                    "test_start": test_start,
                    "test_end": test_end,
                    "true_train_start": episode_start,
                    "true_test_start": train_end,
                    "train_df": train_df,
                    "test_df": test_df
                })

            current_start = test_end

    def visualize():
        display_df = pd.DataFrame([{
            "symbol": w["symbol"],
            "train_start": w["train_start"],
            "true_train_start": w["true_train_start"],
            "train_end": w["train_end"],
            "test_start": w["test_start"],
            "test_end": w["test_end"],
             "true_train_start": episode_start,
            "true_test_start": train_end,
            "train_days": len(w["train_df"]),
            "test_days": len(w["test_df"])
        } for w in windows])
       
        tools.display_dataframe_to_user(name="Walkforward Windows (Buffered)", dataframe=display_df)

    return windows, visualize

from scipy.stats import entropy, kurtosis
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd

# Utility functions
def rolling_volatility(series: pd.Series, window: int = 10):
    return series.rolling(window).std()

def rolling_momentum(series: pd.Series, window: int = 10):
    return series.diff(window)

def calculate_hurst_exponent(ts: np.ndarray, max_lag: int = 20):
    lags = range(2, max_lag)
    tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]
    poly = np.polyfit(np.log(lags), np.log(tau), 1)
    return poly[0]

def calculate_chaos_metrics(df: pd.DataFrame) -> dict:
    price = df['close'].values
    returns = df['close'].pct_change().dropna()

    return {
        "std": np.std(returns),
        "kurt": kurtosis(returns),
        "entropy": entropy(np.histogram(returns, bins=10)[0] + 1),
        "hurst": calculate_hurst_exponent(price),
        "adf_pval": adfuller(returns)[1] if len(returns) > 10 else np.nan,
    }

def extract_features_per_window(windows: List[dict], feature_cols=BASIC_FEATURES) -> pd.DataFrame:
    feature_rows = []

    for w in windows:
        train_df = w['train_df'].copy()
        test_df = w['test_df'].copy()
        train_df['Close']= train_df['close']
        test_df['Close']= test_df['close']
        meta = {
            "symbol": w['symbol'],
            "train_start": w['train_start'],
            "train_end": w['train_end'],
            "test_start": w['test_start'],
            "test_end": w['test_end'],
        }

        # === Chaos features from training window ===
        chaos_features = calculate_chaos_metrics(train_df)
        meta.update(chaos_features)

        # === Rolling Features ===
        window_size = 5 if len(train_df) >= ROLLING else max(2, len(train_df))

        train_df['volatility'] = train_df['close'].rolling(window_size).std()
        train_df['momentum'] = train_df['close'].diff(window_size)

        test_df['volatility'] = test_df['close'].rolling(window_size).std()
        test_df['momentum'] = test_df['close'].diff(window_size)

        # Fallback if iloc[-1] is NaN
        meta["volatility"] = train_df['volatility'].dropna().iloc[-1] if train_df['volatility'].dropna().any() else 0.0
        meta["momentum"] = train_df['momentum'].dropna().iloc[-1] if train_df['momentum'].dropna().any() else 0.0

        scaler = RobustScaler()
     
        train_df.dropna(inplace=True)
        test_df.dropna(inplace=True)
     
        train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols])
        test_df[feature_cols] = scaler.transform(test_df[feature_cols])
        #print(train_df.iloc[40]['date'])
        # === Patch into original dicts ===
        w['train_df'] = train_df.dropna()
        w['test_df'] = test_df.dropna()
        # === Patch into original dicts ===
        #w['train_df'] = train_df.dropna(0)
        #w['test_df'] = test_df.dropna(0)
        
        #scaler = RobustScaler()
        feature_rows.append(meta)

    feature_df = pd.DataFrame(feature_rows)

    def visualize():
        tools.display_dataframe_to_user(name="Meta-Features per Stock Episode", dataframe=feature_df)

    return feature_df, visualize



In [270]:
# Execute on walkforward windows
#feature_df = extract_features_per_window(walkforward_windows,    max_feature_rolling_window=ROLLING,lookback_buffer_days=LOOKBACK)

df = OHLCV_DF[OHLCV_DF['symbol']=="AAPL"].copy()
walkforward_windows, visualize_walkforward_windows = generate_walkforward_windows(df ,max_feature_rolling_window=ROLLING,lookback_buffer_days=LOOKBACK)
feature_df,visualize_feature_df = extract_features_per_window(walkforward_windows,feature_cols=BASIC_FEATURES)
visualize_walkforward_windows()
visualize_feature_df()

Walkforward Windows (Buffered)


Loading ITables v2.4.0 from the internet... (need help?)


Meta-Features per Stock Episode


Loading ITables v2.4.0 from the internet... (need help?)


### 🤖 2. **Agent Training per Episode**

Train **PPO** and optionally **A2C** using:

* Our `RecurrentPPO + TransformerPolicy`
* `SequenceAwareNormAbsMoveEnv`
* Early stopping if reward plateaus
* Use `Monitor` wrapper to track episode stats


In [202]:
# SequenceAwareNormAbsMoveEnv for walkforward context + full training episode
import gym
import numpy as np
import pandas as pd
from gym import spaces

class SequenceAwareNormAbsMoveEnv(gym.Env):
    def __init__(self, df: pd.DataFrame,feature_cols=BASIC_FEATURES, context_window: int = 20, seed: int = 42, episode_steps:int=100):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.context_window = context_window
        self.seed_value = seed
        self.rng = np.random.default_rng(seed)

        self.feature_cols = feature_cols#[col for col in self.df.columns if col not in ['date', 'symbol']]
        self.feature_dim = len(self.feature_cols)
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.context_window, self.feature_dim), dtype=np.float32
        )

        # Reward normalization
        self.reward_scale = 1.0
        self.reward_centering = True
        self._price_diffs = self.df['close'].diff().fillna(0)

        # Indices
        self.start_index = self.context_window
        self.end_index = len(self.df)
        self.current_index = self.start_index

    def reset(self):
        self.start_index = self.context_window
        self.current_index = self.start_index
        self.step_counter = 0
        self.max_steps = self.end_index - self.start_index
        return self._get_observation()

    def _get_observation(self):
        obs_window = self.df.iloc[self.current_index - self.context_window:self.current_index][self.feature_cols]
        return obs_window.values.astype(np.float32)

    def step(self, action):
        reward = self._compute_reward(action)
        self.current_index += 1
        self.step_counter += 1
        done = self.current_index >= self.end_index

        obs = self._get_observation() if not done else np.zeros((self.context_window, self.feature_dim), dtype=np.float32)
        return obs, reward, done, {}

    def _compute_reward(self, action):
        price_change = self._price_diffs.iloc[self.current_index] if self.current_index > 0 else 0
        direction = 0 if action == 0 else (1 if action == 1 else -1)
        reward = direction * price_change
        if self.reward_centering:
            reward -= self._price_diffs.mean()
        return reward * self.reward_scale

    def seed(self, seed=None):
        self.seed_value = seed
        self.rng = np.random.default_rng(seed)


In [203]:
# Transformer Feature Extractor ======================================
import torch
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

MAX_LEN = 64

def generate_causal_mask(seq_len):
    return torch.triu(torch.ones((seq_len, seq_len), dtype=torch.bool), diagonal=1)

class TransformerFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, d_model=64, n_heads=4, n_layers=2, max_len=MAX_LEN):
        super().__init__(observation_space, features_dim=d_model)
        self.d_model = d_model
        input_dim = observation_space.shape[-1]

        self.input_proj = nn.Sequential(
            nn.LayerNorm(input_dim),
            nn.Linear(input_dim, d_model),
            nn.ReLU()
        )
        self.positional_encoding = nn.Parameter(torch.zeros(max_len, d_model))
        nn.init.normal_(self.positional_encoding, std=0.02)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, batch_first=True, norm_first=True, dropout=0.1
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

    def forward(self, obs):
        x = self.input_proj(obs)
        seq_len = x.size(1)
        x = x + self.positional_encoding[:seq_len]
        mask = generate_causal_mask(seq_len).to(x.device)
        x = self.transformer(x, mask=mask)
        return x[:, -1]


# Transformer Policy ================================================
from sb3_contrib.common.recurrent.policies import RecurrentActorCriticPolicy

class TransformerPolicy(RecurrentActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            **kwargs,
            features_extractor_class=TransformerFeatureExtractor,
            features_extractor_kwargs=dict(
                d_model=64, n_heads=4, n_layers=2, max_len=MAX_LEN
            ),
            share_features_extractor=True  # ✅ Ensures SB3_contrib uses recurrent interface correctly
        )


### 🏷️ 3. **Label Advantage for Predictability**

Evaluate each stock-month:

* Compute agent reward vs. random agent:

  $$
  \text{Advantage} = \mathbb{E}[R_{\text{agent}} - R_{\text{random}}]
  $$
* Run **t-test** and **Mann-Whitney U test**:

  * `p_value < 0.05` ⇒ statistically significant edge
  * Log these in the metadata

---


In [204]:
import numpy as np
from scipy.stats import ttest_ind, mannwhitneyu

def compute_advantage_statistics(agent_rewards, random_rewards, alpha=0.05):
    """
    Compute the statistical advantage of the agent over a random policy.

    Parameters:
        agent_rewards: List or np.array of total rewards from trained agent
        random_rewards: List or np.array of total rewards from random actions
        alpha: Significance threshold (default = 0.05)

    Returns:
        Dictionary with advantage stats and significance flag
    """
    agent_rewards = np.array(agent_rewards)
    random_rewards = np.array(random_rewards)

    advantage = agent_rewards.mean() - random_rewards.mean()
    t_stat, t_pval = ttest_ind(agent_rewards, random_rewards, equal_var=False)
    mw_stat, mw_pval = mannwhitneyu(agent_rewards, random_rewards, alternative='greater')

    return {
        "advantage": advantage,
        "agent_mean": agent_rewards.mean(),
        "random_mean": random_rewards.mean(),
        "t_stat": t_stat,
        "t_pval": t_pval,
        "mw_stat": mw_stat,
        "mw_pval": mw_pval,
        "significant": (t_pval < alpha) and (mw_pval < alpha)
    }


### ⚠️ 4. **Only Keep Statistically Significant Runs**

* Filter episodes where agent significantly beats random.
* Label those stock-months as **"predictable."**
* Store mean/variance/confidence intervals for advantage.

---


In [205]:
import os
import json
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

def evaluate_episode_significance(episode_id, agent_rewards, random_rewards, alpha=0.05):
    """Compute statistical tests and determine predictability."""
    t_stat, p_val_t = ttest_ind(agent_rewards, random_rewards, equal_var=False)
    u_stat, p_val_mw = mannwhitneyu(agent_rewards, random_rewards, alternative='two-sided')

    advantage = np.mean(agent_rewards) - np.mean(random_rewards)
    result = {
        "episode_id": episode_id,
        "advantage_mean": advantage,
        "agent_mean": np.mean(agent_rewards),
        "agent_std": np.std(agent_rewards),
        "random_mean": np.mean(random_rewards),
        "random_std": np.std(random_rewards),
        "ttest_pval": p_val_t,
        "mw_pval": p_val_mw,
        "predictable": (p_val_t < alpha) and (p_val_mw < alpha)
    }
    return result

def evaluate_all_episodes(agent_data_dict, save_path=None, alpha=0.05):
    """
    Evaluate multiple episodes for statistical significance and optionally save results.

    agent_data_dict: {
        episode_id: {
            'agent': [...],
            'random': [...]
        }
    }
    """
    results = []
    for episode_id, data in agent_data_dict.items():
        agent_rewards = data['agent']
        random_rewards = data['random']
        result = evaluate_episode_significance(episode_id, agent_rewards, random_rewards, alpha=alpha)
        results.append(result)

    df = pd.DataFrame(results)
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        df.to_csv(save_path, index=False)
    return df


In [274]:
import pandas as pd
import numpy as np
from typing import List, Dict
from scipy.stats import ttest_ind, mannwhitneyu

from stable_baselines3.common.vec_env import DummyVecEnv
from sb3_contrib.ppo_recurrent import RecurrentPPO



def compute_advantage_statistics(agent_rewards, random_rewards, alpha=0.05):
    t_stat, p_val_t = ttest_ind(agent_rewards, random_rewards, equal_var=False)
    u_stat, p_val_mw = mannwhitneyu(agent_rewards, random_rewards, alternative='two-sided')
    advantage = np.mean(agent_rewards) - np.mean(random_rewards)

    return {
        "agent_mean": np.mean(agent_rewards),
        "agent_std": np.std(agent_rewards),
        "random_mean": np.mean(random_rewards),
        "random_std": np.std(random_rewards),
        "advantage": advantage,
        "ttest_pval": p_val_t,
        "mw_pval": p_val_mw,
        "predictable": (p_val_t < alpha) and (p_val_mw < alpha)
    }


def run_walkforward_pipeline(windows: List[Dict],
                             policy_class,
                             seeds=SEEDS,
                             steps: int = 200,
                             context_window: int = 20,
                             episode_steps: int = 21,
                             alpha: float = 0.05) -> pd.DataFrame:
    """
    For each stock-month window:
    - Trains agents
    - Collects rewards vs random
    - Computes statistical advantage
    - Filters significant episodes
    - Returns labeled dataframe
    """
    records = []

    for window in [windows[0]]:
        symbol = window['symbol']
        train_df = window['train_df']
        test_df = window['test_df']
        train_start = window['train_start']
        train_end = window['train_end']
        test_start = window['test_start']
        test_end = window['test_end']

        agent_rewards = []
        random_rewards = []

        for seed in [seeds[0]]:
            print(f"[{symbol}] Seed {seed} — training agent")

            # ==== Train PPO Agent ====
            env_fn = lambda: SequenceAwareNormAbsMoveEnv(train_df, context_window=context_window,
                                                          episode_steps=episode_steps, seed=seed)
            env = DummyVecEnv([env_fn])
            model = RecurrentPPO(
                policy_class,
                env,
                verbose=1,
                seed=seed,
                n_steps=128,
                policy_kwargs=dict(
                    lstm_hidden_size=64,
                )
            )
            model.learn(total_timesteps=steps)

            # ==== Evaluate PPO Agent ====
            test_env = SequenceAwareNormAbsMoveEnv(test_df, context_window=context_window,
                                                   episode_steps=episode_steps, seed=seed)
            obs = test_env.reset()
            print("obs shape:", obs.shape)
            print("expected shape:", model.observation_space.shape)
            done = False
            total_reward = 0
            lstm_states = None
            episode_starts = np.ones((1,), dtype=bool)

            while not done:
                action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True)
                obs, reward, done, _ = test_env.step(action)
                total_reward += reward
                episode_starts = np.zeros((1,), dtype=bool)
            agent_rewards.append(total_reward)

            # ==== Random Policy ====
            test_env = SequenceAwareNormAbsMoveEnv(test_df, context_window=context_window,
                                                   episode_steps=episode_steps, seed=seed)
            obs = test_env.reset()
            done = False
            total_reward = 0

            while not done:
                action = test_env.action_space.sample()
                obs, reward, done, _ = test_env.step(action)
                total_reward += reward
            random_rewards.append(total_reward)

        # ==== Stats & Label ====
        print('x')
        stats = compute_advantage_statistics(agent_rewards, random_rewards, alpha)
        stats.update({
            "symbol": symbol,
            "train_start": train_start,
            "train_end": train_end,
            "test_start": test_start,
            "test_end": test_end
        })
        records.append(stats)

    df = pd.DataFrame(records)
    df.to_csv("advantage_labels_predictability.csv", index=False)
    return df


In [ ]:
run_walkforward_pipeline(walkforward_windows,TransformerPolicy, context_window=LOOKBACK)

[AAPL] Seed 66923877 — training agent
Using cpu device


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


----------------------------
| time/              |     |
|    fps             | 252 |
|    iterations      | 1   |
|    time_elapsed    | 0   |
|    total_timesteps | 128 |
----------------------------


In [271]:
test_df = walkforward_windows[0]['train_df']
test_env = SequenceAwareNormAbsMoveEnv(test_df, context_window=40,
                                                   episode_steps=20, seed=1)

In [272]:
test_env.reset().shape

(40, 11)

In [273]:
test_env.df.iloc[40]

index                                   33673
id                                      33674
symbol                                   AAPL
timestamp                 2023-01-03 05:00:00
date                      2023-01-03 00:00:00
open                                   130.28
high                                    130.9
low                                    124.17
close                                -1.38795
volume                               2.260277
trade_count                         1021067.0
vwap                               125.660032
weekday                                   1.0
day_of_month                                3
day_of_week                                 1
candle_size                              6.73
order_flow                          -1.562455
candle_body                          0.500351
upper_shadow                        -0.431747
lower_shadow                        -0.244813
price_change                         -1.48388
candle_change                     